In [106]:
!az login --tenant 16b3c013-d300-468d-ac64-7eda0820b6d3

ERROR: User cancelled the Accounts Control Operation.. Status: Response_Status.Status_UserCanceled, Error code: 0, Tag: 528315210
Please explicitly log in with:
az login


In [107]:
!az configure --defaults workspace=gpu-workspace group=antonslutsky-rg

In [108]:
%%writefile ./dependencies/job.yml
$schema: https://azuremlschemas.azureedge.net/latest/commandJob.schema.json
code: ../src
command: >-
  python mae_trainer.py 
  --dataset_directory_name ${{inputs.data}} 
  --trainset ${{inputs.trainset}} 
  --valset ${{inputs.valset}} 
  --num_epochs ${{inputs.num_epochs}} 
  --batch_size ${{inputs.batch_size}} 
  --num_workers ${{inputs.num_workers}}  
  --num_nodes ${{inputs.num_nodes}} 
  --num_devices ${{inputs.num_devices}}  
  --strategy ${{inputs.strategy}} 
  --img_size ${{inputs.img_size}} 
  --learning_rate ${{inputs.learning_rate}} 
  --weight_decay ${{inputs.weight_decay}}
  --output_dir ${{outputs.output_dir}}
inputs:
  data:
    type: uri_folder
    path: azureml://subscriptions/781b03e7-6eb7-4506-bab8-cf3a0d89b1d4/resourcegroups/antonslutsky-rg/workspaces/gpu-workspace/datastores/workspaceblobstore/paths/UI/2023-12-05_185853_UTC/ImageNet_1000_sm/imagenet-mini/
    mode: ro_mount      
  trainset:
    type: uri_file
    path: azureml://subscriptions/781b03e7-6eb7-4506-bab8-cf3a0d89b1d4/resourcegroups/antonslutsky-rg/workspaces/gpu-workspace/datastores/workspaceblobstore/paths/UI/2023-12-05_190730_UTC/ImageNet_1000_train.csv
    mode: ro_mount
  valset:
    type: uri_file
    path: azureml://subscriptions/781b03e7-6eb7-4506-bab8-cf3a0d89b1d4/resourcegroups/antonslutsky-rg/workspaces/gpu-workspace/datastores/workspaceblobstore/paths/UI/2023-12-05_190801_UTC/ImageNet_1000_val.csv
    mode: ro_mount        
  num_epochs: 1
  batch_size: 12
  num_workers: 10
  num_nodes: 4
  num_devices: 1
  #strategy: deepspeed_stage_3
  strategy: "deepspeed"
  #strategy: set_strategy(args)
  img_size: 224
  learning_rate: 5e-4
  weight_decay: 0.05
outputs:
  output_dir:
    type: uri_folder
    mode: upload
environment: azureml:godzilla-environment@latest
resources:
  instance_count: 4
distribution:
  type: pytorch 
  process_count_per_instance: 1
compute: azureml:four-nodes
display_name: mae_trainer
experiment_name: mae_trainer
description: Train a Machine Learning model using a workspace Data asset.

Overwriting ./dependencies/job.yml


In [109]:
!az ml job create -f dependencies/job.yml

{
  "code": "azureml:/subscriptions/781b03e7-6eb7-4506-bab8-cf3a0d89b1d4/resourceGroups/antonslutsky-rg/providers/Microsoft.MachineLearningServices/workspaces/gpu-workspace/codes/f5c2780d-64a5-42a8-9c00-7ab48e99bc0b/versions/1",
  "command": "python mae_trainer.py  --dataset_directory_name ${{inputs.data}}  --trainset ${{inputs.trainset}}  --valset ${{inputs.valset}}  --num_epochs ${{inputs.num_epochs}}  --batch_size ${{inputs.batch_size}}  --num_workers ${{inputs.num_workers}}   --num_nodes ${{inputs.num_nodes}}  --num_devices ${{inputs.num_devices}}   --strategy ${{inputs.strategy}}  --img_size ${{inputs.img_size}}  --learning_rate ${{inputs.learning_rate}}  --weight_decay ${{inputs.weight_decay}} --output_dir ${{outputs.output_dir}}",
  "compute": "azureml:four-nodes",
  "creation_context": {
    "created_at": "2024-01-23T15:53:02.117437+00:00",
    "created_by": "Anton Slutsky",
    "created_by_type": "User"
  },
  "description": "Train a Machine Learning model using a workspace Da

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.

Uploading src (0.12 MBs): 100%|######